<a href="https://colab.research.google.com/github/Liming547/Backtest/blob/main/Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kernc/backtesting.py.git
%cd backtesting.py
!pip install -e .
!pip install scikit-optimize

fatal: destination path 'backtesting.py' already exists and is not an empty directory.
/content/backtesting.py
Obtaining file:///content/backtesting.py
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for Backtesting (pyproject.toml) ... done
  Created wheel for Backtesting: filename=Backtesting-0.3.4.dev30+g0ce24d8-0.editable-py3-none-any.whl size=17258 sha256=8c75b45a34b430a326cce4214ed528099d053fc0a23843b370282fc17aa77471
  Stored in directory: /tmp/pip-ephem-wheel-cache-s0i6b8bl/wheels/66/59/b0/47bfa70416155f27f58cf75f0fe73724185d696f4ac6a6c4a5
Successfully built Backtesting
  Attempting uninstall: Backtesting
    Found existing installation: Backtesting 0.3.4.dev30+g0ce24d8
    Uninstalling Backtesting-0.3.4.dev30+g0ce24d8:
      Successfully uninstalled Backtest

In [ ]:
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd

GOOG.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2148 entries, 2004-08-19 to 2013-03-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    2148 non-null   float64
 1   High    2148 non-null   float64
 2   Low     2148 non-null   float64
 3   Close   2148 non-null   float64
 4   Volume  2148 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 100.7 KB


# Mean Reversion Strategy

In [ ]:
def MRS(values, n):
    mean = pd.Series(values).rolling(n).mean()
    std = pd.Series(values).rolling(n).std()
    return mean+2*std, mean-2*std

In [ ]:
class MRSstrategy(Strategy):
    n = 20

    def init(self):
        self.upper, self.lower = self.I(MRS, self.data.Close, self.n)


    def next(self):
        if crossover(self.data.Close, self.upper):
            self.position.close()
            self.buy()


        elif crossover(self.data.Close, self.lower):
            self.position.close()
            self.sell()

In [ ]:
%%time
bt = Backtest(GOOG, MRSstrategy, cash=10_000, commission=.002)
stats = bt.optimize(n=range(5, 30, 1),
                    maximize='Equity Final [$]',
                    method="skopt")
print(stats)
stats._strategy
bt.plot()

Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [9] before, using random point [11]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [9] before, using random point [10]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [9] before, using random point [29]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [9] before, using random point [5]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [9] before, using random point [14]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The 

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   99.255121
Equity Final [$]                  65551.27428
Equity Peak [$]                  142500.89864
Return [%]                         555.512743
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   24.680927
Volatility (Ann.) [%]               43.251907
Sharpe Ratio                         0.570632
Sortino Ratio                        1.102435
Calmar Ratio                         0.354025
Max. Drawdown [%]                   -69.71517
Avg. Drawdown [%]                   -6.276577
Max. Drawdown Duration     1100 days 00:00:00
Avg. Drawdown Duration       45 days 00:00:00
# Trades                                   99
Win Rate [%]                        57.575758
Best Trade [%]                      34.379278
Worst Trade [%]                    -34.252589
Avg. Trade [%]                    

/content/backtesting.py/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/content/backtesting.py/backtesting/_plotting.py:660: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/content/backtesting.py/backtesting/_plotting.py:660: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


CPU times: user 5.46 s, sys: 156 ms, total: 5.61 s
Wall time: 5.62 s


GridPlot(id='p1329', ...)


## Moving Average Crossover Strategy


In [ ]:
def SMA(values, n):
    return pd.Series(values).rolling(n).mean()

In [ ]:
class SmaCross(Strategy):
    n1 = 10 # short MA
    n2 = 20 # long MA

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
%%time
bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats = bt.optimize(n1=range(5, 30, 2),
                    n2=range(10, 70, 2),
                    maximize='Equity Final [$]',
                    method="skopt",
                    constraint=lambda param: param.n1 < param.n2)
print(stats)
stats._strategy
bt.plot()

Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [20, 10] before, using random point [7, 60]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [21, 13] before, using random point [19, 58]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [15, 52] before, using random point [9, 64]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [17, 11] before, using random point [23, 47]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [27, 14] before, using random point [16, 41]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/o

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   99.068901
Equity Final [$]                  125524.3624
Equity Peak [$]                   133123.3106
Return [%]                        1155.243624
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   34.555376
Volatility (Ann.) [%]               44.091069
Sharpe Ratio                         0.783727
Sortino Ratio                        1.699682
Calmar Ratio                         0.800136
Max. Drawdown [%]                  -43.186864
Avg. Drawdown [%]                   -6.187567
Max. Drawdown Duration      560 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                  123
Win Rate [%]                        51.219512
Best Trade [%]                      59.725007
Worst Trade [%]                    -19.357954
Avg. Trade [%]                    

/content/backtesting.py/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/content/backtesting.py/backtesting/_plotting.py:660: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/content/backtesting.py/backtesting/_plotting.py:660: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


CPU times: user 35.1 s, sys: 323 ms, total: 35.4 s
Wall time: 36.8 s


GridPlot(id='p1690', ...)

# Momentum Strategy

In [ ]:
def Momentum(values, n):
    delta = pd.Series(values).diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(n).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(n).mean()
    rs = gain / loss  # Relative strength
    rsi = 100 - (100 / (1 + rs))  # Relative Strength Index
    return rsi


In [ ]:
class Momentumstrategy(Strategy):
    n = 10

    def init(self):
        self.rsi = self.I(Momentum, self.data.Close, self.n)

    def next(self):
        if self.rsi < 30:
            self.position.close()
            self.sell()

        elif self.rsi > 70:
            self.position.close()
            self.buy()

In [ ]:
%%time
bt = Backtest(GOOG, Momentumstrategy, cash=10_000, commission=.002)
stats = bt.optimize(n=range(3, 30, 1),
                    maximize='Equity Final [$]',
                    method="grid")
print(stats)
stats._strategy
bt.plot()

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   99.068901
Equity Final [$]                  12743.50332
Equity Peak [$]                   43327.59236
Return [%]                          27.435033
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    2.885062
Volatility (Ann.) [%]               36.219583
Sharpe Ratio                         0.079655
Sortino Ratio                        0.120577
Calmar Ratio                         0.039158
Max. Drawdown [%]                  -73.677195
Avg. Drawdown [%]                   -8.279363
Max. Drawdown Duration     1558 days 00:00:00
Avg. Drawdown Duration       87 days 00:00:00
# Trades                                  806
Win Rate [%]                        54.218362
Best Trade [%]                      21.724957
Worst Trade [%]                    -25.989776
Avg. Trade [%]                    

/content/backtesting.py/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/content/backtesting.py/backtesting/_plotting.py:660: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/content/backtesting.py/backtesting/_plotting.py:660: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


CPU times: user 1.06 s, sys: 85.2 ms, total: 1.14 s
Wall time: 7.13 s


GridPlot(id='p5561', ...)